<a href="https://colab.research.google.com/github/MikhailMakhnev/Workshop/blob/main/LinkedIn_Works_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LinkedIn рестарт




1.   Открытие файла и изучение инофрмации
2.   Предобработка данных
3.   Удаление неныжный атрибутов
4.   Подгатовка списка навыков
5.   Заключение



### 1. Откроем файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
import re
pd.set_option('display.max_columns', None) # опция для вывода всех колонок

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/linkedin.csv')

In [ ]:
data.head(5)

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            998 non-null    object 
 1   location         998 non-null    object 
 2   country          998 non-null    object 
 3   employment_type  998 non-null    object 
 4   company_name     996 non-null    object 
 5   employee_qty     998 non-null    object 
 6   company_field    998 non-null    object 
 7   skills           998 non-null    object 
 8   job_description  998 non-null    object 
 9   applicants       838 non-null    float64
dtypes: float64(1), object(9)
memory usage: 78.1+ KB


## 2. Предобработка данных

### 2.1. Первое, что мы сделаем - это отфельтруем данные с нужными вакансиями

In [ ]:
data = data[data['title'].str.contains('Data Analyst|Data Scientist|DA|DS', case=False)]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 686 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            686 non-null    object 
 1   location         686 non-null    object 
 2   country          686 non-null    object 
 3   employment_type  686 non-null    object 
 4   company_name     684 non-null    object 
 5   employee_qty     686 non-null    object 
 6   company_field    686 non-null    object 
 7   skills           686 non-null    object 
 8   job_description  686 non-null    object 
 9   applicants       566 non-null    float64
dtypes: float64(1), object(9)
memory usage: 59.0+ KB


мы выбрали вакаснии для аналитиков данных

### 2.2. Проверка на явные дубликаты

In [ ]:
data.duplicated().sum()

108

Удалим явные дублика из дата фрейма

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.head(10)

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN
5,Data Analyst - Logistics,Southampton,United Kingdom,On-site,"Butler, Bridge & May",not specified,not specified,None,Location: Southam...,NaN
6,Data Analyst,Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,not specified,not specified,None,We’re Maria Malla...,NaN
7,Data Analyst,Nuneaton,United Kingdom,Hybrid,Kelly Group,not specified,not specified,None,Kelly Group are s...,NaN
8,Data Analyst,Paris,France,On-site,eXalt,"501-1,000 employees",IT Services and IT Consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",Qui sont-ils ? ...,140.0
9,Data Analyst - Hybrid Working,Cambridge,United Kingdom,On-site,Blue Arrow,not specified,not specified,None,Data Analyst ...,NaN


### 2.3. Проверим значение сталбцов

#### 2.3.1 Столбец "Тип занятости"


In [ ]:
data['employment_type'].unique()

array(['On-site', 'Hybrid', 'not specified', 'Remote'], dtype=object)

In [ ]:
print (data.query('employment_type == "not specified"')['title'].count(), 'вакансий без указаия типа занятости')

48 вакансий без указаия типа занятости


In [ ]:
data['employment_type'] = data['employment_type'].str.replace('Remote', 'Online')

#### 2.3.2 Обработка хард скилов

In [ ]:
skills = [
    'a/b testing', 'ab testing', 'actian', 'adobe analytics', 'adobe audience manager',
    'adobe experience platform', 'adobe launch', 'adobe target', 'ai', 'airflow',
    'alooma', 'alteryx', 'amazon machine learning', 'amazon web services', 'aml',
    'amplitude', 'ansible', 'apache camel', 'apache nifi', 'apache spark',
    'api', 'asana', 'auth0', 'aws', 'aws glue', 'azure', 'azure data factory',
    'basecamp', 'bash', 'beats', 'big query', 'bigquery', 'birst', 'bitbucket',
    'blendo', 'bootstrap', 'business objects bi', 'c#', 'c++', 'caffe', 'cassandra',
    'cdata sync', 'chronograf', 'ci/cd', 'cicd', 'clickhouse', 'cloudera', 'cluvio',
    'cntk', 'cognos', 'composer', 'computer vision', 'conda', 'confluence',
    'couchbase', 'css', 'd3.js', 'dash', 'dashboard', 'data factory', 'data fusion',
    'data mining', 'data studio', 'data warehouse', 'databricks', 'dataddo',
    'dataflow', 'datahub', 'dataiku', 'datastage', 'dbconvert', 'dbeaver', 'dbt',
    'deep learning', 'dl/ml', 'docker', 'domo', 'dune', 'dv360', 'dynamodb',
    'elasticsearch', 'elt', 'erwin', 'etl', 'etleap', 'excel', 'facebook business manager',
    'fivetran', 'fuzzy', 'ga360', 'gcp', 'gensim', 'ggplot', 'git', 'github', 'gitlab',
    'google ads', 'google analytics', 'google cloud platform', 'google data flow',
    'google optimize', 'google sheets', 'google tag manager', 'google workspace',
    'grafana', 'hadoop', 'hana', 'hanagrafana', 'hbase', 'hdfs', 'hevo data', 'hightouch',
    'hive', 'hivedatabricks', 'html', 'hubspot', 'ibm coremetrics', 'inetsoft',
    'influxdb', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java',
    'java script', 'javascript', 'jenkins', 'jira', 'jmp', 'julia', 'jupyter',
    'k2view', 'kafka', 'kantar', 'kapacitor', 'keras', 'kibana', 'kubernetes',
    'lambda', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab',
    'matplotlib', 'mendix', 'metabase', 'microsoft sql', 'microsoft sql server',
    'microstrategy', 'miro', 'mixpanel', 'ml', 'ml flow', 'mlflow', 'mongodb', 'mxnet',
    'mysql', 'natural nanguage processing', 'neo4j', 'nlp', 'nltk', 'nosql', 'numpy',
    'oauth', 'octave', 'omniture', 'omnituregitlab', 'openshift', 'openstack',
    'optimizely', 'oracle', 'oracle business intelligence', 'oracle data integrator',
    'pandas', 'panorama', 'pentaho', 'plotly', 'postgre', 'postgresql', 'posthog',
    'power amc', 'power bi', 'power point', 'powerbi', 'powerpivot', 'powerpoint',
    'powerquery', 'pyspark', 'python', 'pytorch', 'pytorchhevo data', 'qlik',
    'qlik sense', 'qlikview', 'querysurge', 'r', 'raphtory', 'rapidminer', 'redash',
    'redis', 'redshift', 'retool', 'rivery', 'rust', 's3', 'sa360', 'salesforce', 'sap',
    'sap business objects', 'sas', 'sas visual analytics', 'scala', 'scikit-learn',
    'scipy', 'seaborn', 'segment', 'selenium', 'sem rush', 'semrush', 'shell', 'shiny',
    'singer', 'sisense', 'skyvia', 'snowflake', 'spacy', 'spark', 'sparkml', 'splunk',
    'spotfire', 'spreadsheet', 'spss', 'sql', 'ssis', 'sssr', 'stambia', 'statistics',
    'statsbot', 'stitch', 'streamlit', 'streamsets', 'svn', 't-sql', 'tableau', 'talend',
    'targit', 'tealium', 'telegraf', 'tensorflow', 'terraapi', 'terraform', 'theano',
    'thoughtspot', 'timeseries', 'trello', 'unix', 'vba', 'vtom', 'webfocus', 'wfh',
    'xplenty', 'xtract.io', 'yellowfin'
]

In [ ]:
def skills_finder(cell, skill_list_2=skills):
    matched_skills_list=[]
    for i in skill_list_2:
        if i == 'c++':
          if re.search('\Wc\+\+\W', cell.lower()):
            matched_skills_list.append(i)
        # word_border + rewritten "i" in special symbols + word_border
        else:
          pattern = (
            r'(\b|\W)'
            + re.escape(i) # escape(i)
            + r'(\b|\W)'
            +'|'
            + r'(\b|\W)'
            +re.escape(i.replace(' ', ''))
            + r'(\b|\W)'
        )
          if re.search(pattern, cell.lower()):
              matched_skills_list.append(i)
    return matched_skills_list

In [ ]:
data['hard_skills'] = data['job_description'].apply(skills_finder)

In [ ]:
data.head(10)

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,hard_skills
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0,"[data mining, excel, sap, sas, spss, sql, stat..."
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN,[]
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN,[wfh]
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN,[excel]
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN,"[aws, data warehouse, etl, gcp, oracle, oracle..."
5,Data Analyst - Logistics,Southampton,United Kingdom,On-site,"Butler, Bridge & May",not specified,not specified,None,Location: Southam...,NaN,"[excel, power bi, powerbi, sap]"
6,Data Analyst,Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,not specified,not specified,None,We’re Maria Malla...,NaN,"[excel, power bi]"
7,Data Analyst,Nuneaton,United Kingdom,Hybrid,Kelly Group,not specified,not specified,None,Kelly Group are s...,NaN,"[data mining, excel]"
8,Data Analyst,Paris,France,On-site,eXalt,"501-1,000 employees",IT Services and IT Consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",Qui sont-ils ? ...,140.0,"[dataiku, nosql, power bi, powerbi, qlikview, ..."
9,Data Analyst - Hybrid Working,Cambridge,United Kingdom,On-site,Blue Arrow,not specified,not specified,None,Data Analyst ...,NaN,[]


### 2.4 Исправления столбца Страны

In [ ]:
data['country'].unique()

array([' Switzerland', ' United Kingdom', ' France', ' Netherlands',
       ' Ireland', ' Greece', ' Sweden', ' Lithuania', ' Belgium',
       ' Luxembourg', ' Germany', ' Italy', 'Brussels Metropolitan Area',
       'Greater Paris Metropolitan Region', 'France', ' Spain', 'Hungary',
       'Greater Palma de Mallorca Metropolitan Area', ' Poland',
       ' Czechia', ' Hungary', 'Wroclaw Metropolitan Area', 'Poland',
       ' Portugal', ' Latvia', ' Estonia',
       'Greater Barcelona Metropolitan Area', ' Malta', ' Austria',
       'Iasi Metropolitan Area', 'Cologne Bonn Region',
       'Greater Munster Area', 'Greater Banska Bystrica Area',
       'Greater Nuremberg Metropolitan Area', 'Berlin Metropolitan Area',
       'Germany', ' Romania', ' Norway', 'Rotterdam and The Hague',
       'Prague Metropolitan Area', 'Denmark', ' Bulgaria',
       'Greater Pau Area', ' Monaco', 'Lisbon Metropolitan Area',
       'Athens Metropolitan Area', 'Italy', ' Croatia',
       'Stuttgart Region', 

Потребуется вручную исправить страны, где записаны города или локации

In [ ]:
data['country'] = data['country'].str.replace('Brussels Metropolitan Area', 'Belgium')
data['country'] = data['country'].str.replace('Greater Paris Metropolitan Region', 'France')
data['country'] = data['country'].str.replace('Greater Palma de Mallorca Metropolitan Area', 'Spain')
data['country'] = data['country'].str.replace('Wroclaw Metropolitan Area', 'Poland')
data['country'] = data['country'].str.replace('Greater Barcelona Metropolitan Area', 'Spain')
data['country'] = data['country'].str.replace('Iasi Metropolitan Area', 'Romania')
data['country'] = data['country'].str.replace('Cologne Bonn Region', 'Germany')
data['country'] = data['country'].str.replace('Greater Munster Area', 'Germany')
data['country'] = data['country'].str.replace('Greater Banska Bystrica Area', 'Slovakia')
data['country'] = data['country'].str.replace('Greater Nuremberg Metropolitan Area', 'Germany')
data['country'] = data['country'].str.replace('Berlin Metropolitan Area', 'Germany')
data['country'] = data['country'].str.replace('Rotterdam and The Hague', 'Netherlands')
data['country'] = data['country'].str.replace('Prague Metropolitan Area', 'Czechia')
data['country'] = data['country'].str.replace('Greater Pau Area', 'France')
data['country'] = data['country'].str.replace('Lisbon Metropolitan Area', 'Portugal')
data['country'] = data['country'].str.replace('Athens Metropolitan Area', 'Greece')
data['country'] = data['country'].str.replace('Stuttgart Region', 'Germany')
data['country'] = data['country'].str.replace('Greater Milan Metropolitan Area', 'Italy')
data['country'] = data['country'].str.replace('Amsterdam Area', 'Netherlands')
data['country'] = data['country'].str.replace('Eindhoven Area', 'Netherlands')
data['country'] = data['country'].str.replace('Greater Madrid Metropolitan Area', 'Spain')
data['country'] = data['country'].str.replace('Krakow Metropolitan Area', 'Poland')
data['country'] = data['country'].str.replace('Copenhagen Metropolitan Area', 'Denmark')
data['country'] = data['country'].str.replace('Greater Munich Metropolitan Area', 'Germany')

In [ ]:
data['country'] = data['country'].str.replace(' ', '')

Проверка списка

In [ ]:
data['country'].unique()

array(['Switzerland', 'UnitedKingdom', 'France', 'Netherlands', 'Ireland',
       'Greece', 'Sweden', 'Lithuania', 'Belgium', 'Luxembourg',
       'Germany', 'Italy', 'Spain', 'Hungary', 'Poland', 'Czechia',
       'Portugal', 'Latvia', 'Estonia', 'Malta', 'Austria', 'Romania',
       'Slovakia', 'Norway', 'Denmark', 'Bulgaria', 'Monaco', 'Croatia',
       'Finland'], dtype=object)

## 3. Удаление неныжный атрибутов

Удалим из таблицы столбец "applicants", т.к. при визуализации он нам не потребуется

In [ ]:
data = data.drop('applicants', axis=1)
data = data.drop('skills', axis=1)
data = data.drop('job_description', axis=1)

In [ ]:
data.head(5)

,title,location,country,employment_type,company_name,employee_qty,company_field,hard_skills
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,"[data mining, excel, sap, sas, spss, sql, stat..."
1,Data Analyst - Logistics,Coventry,UnitedKingdom,On-site,Resolute Recruitment,not specified,not specified,[]
2,Data Analyst - Logistics,Coventry,UnitedKingdom,On-site,Resolute Recruitment,not specified,not specified,[wfh]
3,Data Analyst (Space & Planning),South Molton,UnitedKingdom,On-site,Mole Valley Farmers,not specified,not specified,[excel]
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,"[aws, data warehouse, etl, gcp, oracle, oracle..."


## 4. Подгатовка списка навыков

In [ ]:
data = data.reset_index()

In [ ]:
data = data.explode('hard_skills')

In [ ]:
data.head(10)

,index,title,location,country,employment_type,company_name,employee_qty,company_field,hard_skills
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,data mining
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,excel
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,sap
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,sas
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,spss
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,sql
0,0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,statistics
1,1,Data Analyst - Logistics,Coventry,UnitedKingdom,On-site,Resolute Recruitment,not specified,not specified,NaN
2,2,Data Analyst - Logistics,Coventry,UnitedKingdom,On-site,Resolute Recruitment,not specified,not specified,wfh
3,3,Data Analyst (Space & Planning),South Molton,UnitedKingdom,On-site,Mole Valley Farmers,not specified,not specified,excel


## 5. Заключение
Произведена подгатовка данных к визуалицаии. Исправлены столбцы с типом занятости и скилами. Добавлен столбец с hard skills. Удалены дубликаты из таблицы. Так же удален столбцы, который не потребуется для постраения дашборда.

In [ ]:
data.to_csv ('/content/drive/MyDrive/my_linkedin.csv', index= False)